In [1]:
from IPython.core.display import display, HTML
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import helper_functions as hf

warnings.filterwarnings('ignore')

In [2]:
prev = pd.read_csv('previous_application.csv').sort_values(['SK_ID_CURR','DAYS_DECISION'])
pos_prev = pd.read_csv('pos_prev.csv')

for i in prev:
    if i.startswith('DAYS'):
        prev[i] = prev[i].replace(365243,np.nan)
    if i.startswith('SELLER'):
        prev[i] = prev[i].replace(-1,np.nan)
        
prev['HOUR_APPR_PROCESS_START'] = hf.standard_scale(prev['HOUR_APPR_PROCESS_START'])

prev['prev_f1'] = prev['AMT_ANNUITY']/prev['AMT_APPLICATION'] 
prev['prev_f2'] = prev['RATE_DOWN_PAYMENT']/prev['RATE_INTEREST_PRIMARY'] 
prev['prev_f3'] = prev['DAYS_LAST_DUE']-prev['DAYS_TERMINATION'] 
prev['prev_f4'] = prev['DAYS_FIRST_DUE']/prev['DAYS_FIRST_DRAWING']

prev_nums = prev.select_dtypes(include=['float64'])
prev_nums['SK_ID_PREV'] = prev['SK_ID_PREV']
prev_nums['RATE_DOWN_PAYMENT'] = prev_nums['RATE_DOWN_PAYMENT'].replace(0,np.nan)
prev_nums = prev_nums.merge(pos_prev,on='SK_ID_PREV',how='left')
prev_nums.head()

,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,SELLERPLACE_AREA,...,SK_DPD_slope,SK_DPD_DEF_slope,NAME_CONTRACT_STATUS_Active_slope,NAME_CONTRACT_STATUS_Approved_slope,NAME_CONTRACT_STATUS_Completed_slope,NAME_CONTRACT_STATUS_Demand_slope,NAME_CONTRACT_STATUS_OTHER_slope,NAME_CONTRACT_STATUS_Returned to the store_slope,NAME_CONTRACT_STATUS_Signed_slope,SK_ID_CURR
0,3951.000,24835.5,23787.0,2520.0,24835.5,NaN,0.104326,NaN,NaN,23.0,...,0.0,0.0,-0.200000,0.0,0.200000,0.0,0.0,0.0,0.0,100001.0
1,9251.775,179055.0,179055.0,0.0,179055.0,NaN,NaN,NaN,NaN,500.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,100002.0
2,6737.310,68809.5,68053.5,6885.0,68809.5,NaN,0.100061,NaN,NaN,200.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,100003.0
3,64567.665,337500.0,348637.5,0.0,337500.0,NaN,NaN,NaN,NaN,1400.0,...,0.0,0.0,-0.083333,0.0,0.083333,0.0,0.0,0.0,0.0,100003.0
4,98356.995,900000.0,1035882.0,NaN,900000.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,-0.083333,0.0,0.083333,0.0,0.0,0.0,0.0,100003.0


In [3]:
prev_cats = prev.select_dtypes(exclude=['float64'])
prev_cats['NAME_CONTRACT_TYPE'] = hf.reduce_cat(prev,'NAME_CONTRACT_TYPE',2,'OTHER')
prev_cats['NAME_CASH_LOAN_PURPOSE'] = hf.reduce_cat(prev,'NAME_CASH_LOAN_PURPOSE',2,'OTHER')
prev_cats['NAME_PAYMENT_TYPE'] = hf.reduce_cat(prev,'NAME_PAYMENT_TYPE',2,'OTHER')
prev_cats['NAME_GOODS_CATEGORY'] = hf.reduce_cat(prev,'NAME_GOODS_CATEGORY',9,'OTHER')
prev_cats['NAME_PORTFOLIO'] = hf.reduce_cat(prev,'NAME_PORTFOLIO',4,'OTHER')
prev_cats['CHANNEL_TYPE'] = hf.reduce_cat(prev,'CHANNEL_TYPE',6,'OTHER')
prev_cats['NAME_SELLER_INDUSTRY'] = hf.reduce_cat(prev,'NAME_SELLER_INDUSTRY',6,'OTHER')
prev_cats['PRODUCT_COMBINATION'] = hf.reduce_cat(prev,'PRODUCT_COMBINATION',7,'OTHER')
prev_cats['NAME_CLIENT_TYPE'] = hf.reduce_cat(prev,'NAME_CLIENT_TYPE',2,'OTHER')
prev_cats['NAME_TYPE_SUITE'] = hf.reduce_cat(prev,'NAME_TYPE_SUITE',4,'OTHER')
prev_cats['CODE_REJECT_REASON'] = hf.reduce_cat(prev,'CODE_REJECT_REASON',5,'OTHER')

prev_cats = pd.get_dummies(prev_cats)
prev = pd.concat([prev_nums.drop('SK_ID_CURR',axis=1).set_index('SK_ID_PREV'),prev_cats.set_index('SK_ID_PREV')],axis=1)
prev.head()

,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,SELLERPLACE_AREA,...,NAME_YIELD_GROUP_low_normal,NAME_YIELD_GROUP_middle,PRODUCT_COMBINATION_Card Street,PRODUCT_COMBINATION_Cash,PRODUCT_COMBINATION_Cash X-Sell: low,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_OTHER,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS mobile with interest
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,
1369693,3951.000,24835.5,23787.0,2520.0,24835.5,NaN,0.104326,NaN,NaN,23.0,...,0,0,0,0,0,0,0,0,0,1
1038818,9251.775,179055.0,179055.0,0.0,179055.0,NaN,NaN,NaN,NaN,500.0,...,1,0,0,0,0,0,1,0,0,0
2396755,6737.310,68809.5,68053.5,6885.0,68809.5,NaN,0.100061,NaN,NaN,200.0,...,0,1,0,0,0,0,0,1,0,0
2636178,64567.665,337500.0,348637.5,0.0,337500.0,NaN,NaN,NaN,NaN,1400.0,...,0,1,0,0,0,0,0,0,1,0
1810518,98356.995,900000.0,1035882.0,NaN,900000.0,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,1,0,0,0,0,0


In [4]:
days = prev['DAYS_DECISION']
prev = hf.drop_high_corrs(prev,0.95)
prev['DAYS_DECISION'] = days

Removing 31 columns ['AMT_CREDIT', 'AMT_GOODS_PRICE', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'CNT_INSTALMENT_max', 'CNT_INSTALMENT_mean', 'CNT_INSTALMENT_FUTURE_max', 'CNT_INSTALMENT_FUTURE_mean', 'CNT_INSTALMENT_FUTURE_var', 'SK_DPD_mean', 'SK_DPD_sum', 'SK_DPD_DEF_mean', 'SK_DPD_DEF_var', 'NAME_CONTRACT_STATUS_Active_mean', 'NAME_CONTRACT_STATUS_Active_var', 'NAME_CONTRACT_STATUS_Approved_var', 'NAME_CONTRACT_STATUS_Completed_mean', 'NAME_CONTRACT_STATUS_Completed_var', 'NAME_CONTRACT_STATUS_Demand_mean', 'NAME_CONTRACT_STATUS_OTHER_mean', 'NAME_CONTRACT_STATUS_OTHER_var', 'NAME_CONTRACT_STATUS_Returned to the store_mean', 'NAME_CONTRACT_STATUS_Signed_mean', 'DAYS_DECISION', 'FLAG_LAST_APPL_PER_CONTRACT_Y', 'NAME_CASH_LOAN_PURPOSE_XAP', 'NAME_PAYMENT_TYPE_XNA', 'CODE_REJECT_REASON_CLIENT', 'NAME_GOODS_CATEGORY_XNA', 'NAME_PORTFOLIO_POS']


In [5]:
prev_aggs_dict = {}

for i in prev.select_dtypes('float64').columns:
    prev_aggs_dict[i] = ['min','max','mean','sum']
    
for i in prev.select_dtypes(exclude='float64').drop('SK_ID_CURR',axis=1).columns:
    prev_aggs_dict[i] = ['mean']

prev_aggs = hf.get_aggs_df(prev,'SK_ID_CURR',prev_aggs_dict)
prev_aggs.head()

100.0 %


,AMT_ANNUITY_min,AMT_ANNUITY_max,AMT_ANNUITY_mean,AMT_ANNUITY_sum,AMT_APPLICATION_min,AMT_APPLICATION_max,AMT_APPLICATION_mean,AMT_APPLICATION_sum,AMT_DOWN_PAYMENT_min,AMT_DOWN_PAYMENT_max,...,NAME_YIELD_GROUP_middle_mean,PRODUCT_COMBINATION_Card Street_mean,PRODUCT_COMBINATION_Cash_mean,PRODUCT_COMBINATION_Cash X-Sell: low_mean,PRODUCT_COMBINATION_Cash X-Sell: middle_mean,PRODUCT_COMBINATION_OTHER_mean,PRODUCT_COMBINATION_POS household with interest_mean,PRODUCT_COMBINATION_POS industry with interest_mean,PRODUCT_COMBINATION_POS mobile with interest_mean,DAYS_DECISION_mean
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3951.000,3951.000,3951.000,3951.000,24835.5,24835.5,24835.50,24835.5,2520.0,2520.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,1.0,-1740.0
100002,9251.775,9251.775,9251.775,9251.775,179055.0,179055.0,179055.00,179055.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,-606.0
100003,6737.310,98356.995,56553.990,169661.970,68809.5,900000.0,435436.50,1306309.5,0.0,6885.0,...,0.666667,0.0,0.0,0.333333,0.0,0.0,0.333333,0.333333,0.0,-1305.0
100004,5357.250,5357.250,5357.250,5357.250,24282.0,24282.0,24282.00,24282.0,4860.0,4860.0,...,1.000000,0.0,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,-815.0
100005,4813.200,4813.200,4813.200,4813.200,0.0,44617.5,22308.75,44617.5,4464.0,4464.0,...,0.000000,0.0,0.5,0.000000,0.0,0.0,0.000000,0.000000,0.5,-536.0


In [6]:
prev_weighted = hf.get_weighted(prev,'SK_ID_CURR','DAYS_DECISION',prev.drop('SK_ID_CURR',axis=1).columns)
prev_weighted.to_csv('prev_weighted.csv',columns = prev_weighted.columns)
prev_weighted = pd.read_csv('prev_weighted.csv').set_index('SK_ID_CURR')
prev_weighted.head()

,AMT_ANNUITY_weighted,AMT_APPLICATION_weighted,AMT_DOWN_PAYMENT_weighted,HOUR_APPR_PROCESS_START_weighted,RATE_DOWN_PAYMENT_weighted,RATE_INTEREST_PRIMARY_weighted,RATE_INTEREST_PRIVILEGED_weighted,SELLERPLACE_AREA_weighted,CNT_PAYMENT_weighted,DAYS_FIRST_DRAWING_weighted,...,NAME_YIELD_GROUP_middle_weighted,PRODUCT_COMBINATION_Card Street_weighted,PRODUCT_COMBINATION_Cash_weighted,PRODUCT_COMBINATION_Cash X-Sell: low_weighted,PRODUCT_COMBINATION_Cash X-Sell: middle_weighted,PRODUCT_COMBINATION_OTHER_weighted,PRODUCT_COMBINATION_POS household with interest_weighted,PRODUCT_COMBINATION_POS industry with interest_weighted,PRODUCT_COMBINATION_POS mobile with interest_weighted,DAYS_DECISION_weighted
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3951.000000,24835.500000,2520.000000,0.0,0.104326,0.0,0.0,23.000000,8.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.000000,-1740.000000
100002,9251.775000,179055.000000,0.000000,0.0,0.000000,0.0,0.0,500.000000,24.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,-606.000000
100003,66693.812160,524264.777218,1501.648050,0.0,0.021824,0.0,0.0,527.582832,9.925877,0.0,...,0.563791,0.0,0.000000,0.436209,0.0,0.0,0.218104,0.345687,0.000000,-1122.222690
100004,5357.250000,24282.000000,4860.000000,0.0,0.212008,0.0,0.0,30.000000,4.000000,0.0,...,1.000000,0.0,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,-815.000000
100005,1861.999932,17260.405127,1726.910931,0.0,0.042153,0.0,0.0,14.313554,4.642234,0.0,...,0.000000,0.0,0.613147,0.000000,0.0,0.0,0.000000,0.000000,0.386853,-485.988941


In [7]:
prev_slopes = hf.get_slopes(prev,prev['SK_ID_CURR'].unique(),
                              'SK_ID_CURR','DAYS_DECISION',prev.drop('SK_ID_CURR',axis=1).columns)
prev_slopes.to_csv('prev_slopes.csv',columns = prev_slopes.columns)
prev_slopes = pd.read_csv('prev_slopes.csv').set_index('SK_ID_CURR')
prev_slopes.head()

99.747 %
2
Found slopes in Index(['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_DOWN_PAYMENT',
       'HOUR_APPR_PROCESS_START', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'SELLERPLACE_AREA', 'CNT_PAYMENT',
       'DAYS_FIRST_DRAWING',
       ...
       'NAME_YIELD_GROUP_middle', 'PRODUCT_COMBINATION_Card Street',
       'PRODUCT_COMBINATION_Cash', 'PRODUCT_COMBINATION_Cash X-Sell: low',
       'PRODUCT_COMBINATION_Cash X-Sell: middle', 'PRODUCT_COMBINATION_OTHER',
       'PRODUCT_COMBINATION_POS household with interest',
       'PRODUCT_COMBINATION_POS industry with interest',
       'PRODUCT_COMBINATION_POS mobile with interest', 'DAYS_DECISION'],
      dtype='object', length=129) grouped by SK_ID_CURR with respect to DAYS_DECISION


,AMT_ANNUITY_slope,AMT_APPLICATION_slope,AMT_DOWN_PAYMENT_slope,HOUR_APPR_PROCESS_START_slope,RATE_DOWN_PAYMENT_slope,RATE_INTEREST_PRIMARY_slope,RATE_INTEREST_PRIVILEGED_slope,SELLERPLACE_AREA_slope,CNT_PAYMENT_slope,DAYS_FIRST_DRAWING_slope,...,NAME_YIELD_GROUP_middle_slope,PRODUCT_COMBINATION_Card Street_slope,PRODUCT_COMBINATION_Cash_slope,PRODUCT_COMBINATION_Cash X-Sell: low_slope,PRODUCT_COMBINATION_Cash X-Sell: middle_slope,PRODUCT_COMBINATION_OTHER_slope,PRODUCT_COMBINATION_POS household with interest_slope,PRODUCT_COMBINATION_POS industry with interest_slope,PRODUCT_COMBINATION_POS mobile with interest_slope,DAYS_DECISION_slope
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
100004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [8]:
prev_curr = pd.concat([prev_aggs, prev_slopes,prev_weighted],axis=1)
prev_curr.head()

,AMT_ANNUITY_min,AMT_ANNUITY_max,AMT_ANNUITY_mean,AMT_ANNUITY_sum,AMT_APPLICATION_min,AMT_APPLICATION_max,AMT_APPLICATION_mean,AMT_APPLICATION_sum,AMT_DOWN_PAYMENT_min,AMT_DOWN_PAYMENT_max,...,NAME_YIELD_GROUP_middle_weighted,PRODUCT_COMBINATION_Card Street_weighted,PRODUCT_COMBINATION_Cash_weighted,PRODUCT_COMBINATION_Cash X-Sell: low_weighted,PRODUCT_COMBINATION_Cash X-Sell: middle_weighted,PRODUCT_COMBINATION_OTHER_weighted,PRODUCT_COMBINATION_POS household with interest_weighted,PRODUCT_COMBINATION_POS industry with interest_weighted,PRODUCT_COMBINATION_POS mobile with interest_weighted,DAYS_DECISION_weighted
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3951.000,3951.000,3951.000,3951.000,24835.5,24835.5,24835.50,24835.5,2520.0,2520.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.000000,-1740.000000
100002,9251.775,9251.775,9251.775,9251.775,179055.0,179055.0,179055.00,179055.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,-606.000000
100003,6737.310,98356.995,56553.990,169661.970,68809.5,900000.0,435436.50,1306309.5,0.0,6885.0,...,0.563791,0.0,0.000000,0.436209,0.0,0.0,0.218104,0.345687,0.000000,-1122.222690
100004,5357.250,5357.250,5357.250,5357.250,24282.0,24282.0,24282.00,24282.0,4860.0,4860.0,...,1.000000,0.0,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,-815.000000
100005,4813.200,4813.200,4813.200,4813.200,0.0,44617.5,22308.75,44617.5,4464.0,4464.0,...,0.000000,0.0,0.613147,0.000000,0.0,0.0,0.000000,0.000000,0.386853,-485.988941


In [10]:
prev_curr.columns = ['prev_'+str(i) for i in prev_curr.columns]
prev_curr.to_csv('prev_curr.csv',columns = prev_curr.columns)
prev_curr = pd.read_csv('prev_curr.csv').set_index('SK_ID_CURR')
prev_curr.head()

,prev_AMT_ANNUITY_min,prev_AMT_ANNUITY_max,prev_AMT_ANNUITY_mean,prev_AMT_ANNUITY_sum,prev_AMT_APPLICATION_min,prev_AMT_APPLICATION_max,prev_AMT_APPLICATION_mean,prev_AMT_APPLICATION_sum,prev_AMT_DOWN_PAYMENT_min,prev_AMT_DOWN_PAYMENT_max,...,prev_NAME_YIELD_GROUP_middle_weighted,prev_PRODUCT_COMBINATION_Card Street_weighted,prev_PRODUCT_COMBINATION_Cash_weighted,prev_PRODUCT_COMBINATION_Cash X-Sell: low_weighted,prev_PRODUCT_COMBINATION_Cash X-Sell: middle_weighted,prev_PRODUCT_COMBINATION_OTHER_weighted,prev_PRODUCT_COMBINATION_POS household with interest_weighted,prev_PRODUCT_COMBINATION_POS industry with interest_weighted,prev_PRODUCT_COMBINATION_POS mobile with interest_weighted,prev_DAYS_DECISION_weighted
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3951.000,3951.000,3951.000,3951.000,24835.5,24835.5,24835.50,24835.5,2520.0,2520.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.000000,-1740.000000
100002,9251.775,9251.775,9251.775,9251.775,179055.0,179055.0,179055.00,179055.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,-606.000000
100003,6737.310,98356.995,56553.990,169661.970,68809.5,900000.0,435436.50,1306309.5,0.0,6885.0,...,0.563791,0.0,0.000000,0.436209,0.0,0.0,0.218104,0.345687,0.000000,-1122.222690
100004,5357.250,5357.250,5357.250,5357.250,24282.0,24282.0,24282.00,24282.0,4860.0,4860.0,...,1.000000,0.0,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,-815.000000
100005,4813.200,4813.200,4813.200,4813.200,0.0,44617.5,22308.75,44617.5,4464.0,4464.0,...,0.000000,0.0,0.613147,0.000000,0.0,0.0,0.000000,0.000000,0.386853,-485.988941
